In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/MyDrive/capstone-2')

In [3]:
!pip install -q -r requirements.txt

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.2/183.2 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 16.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.7/208.7 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 90.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
import pickle
import peft
from peft import LoraConfig, PeftModel
from transformers import AutoTokenizer,BitsAndBytesConfig, AutoModelForCausalLM, CLIPVisionModel, AutoProcessor
import torch
from torch.utils.data import random_split, DataLoader
import pandas as pd
from torch.nn import functional as F
import csv
import random
from PIL import Image
import requests
import wandb
import os
import torch.nn as nn
from finetune_dataset import llavadataset, collate_fn

In [6]:
clip_model_name = "wkcn/TinyCLIP-ViT-61M-32-Text-29M-LAION400M"
phi_model_name  = "microsoft/phi-2"
tokenizer  = AutoTokenizer.from_pretrained(phi_model_name, trust_remote_code=True)
processor  = AutoProcessor.from_pretrained(clip_model_name)
tokenizer.pad_token = tokenizer.eos_token
train_batch_size = 2
clip_embed = 640
phi_embed  = 2560
device = "cuda" if torch.cuda.is_available() else "cpu"
num_workers = 10
IMAGE_TOKEN_ID = 23893 # token for word comment
max_steps      = 10000
EOS_TOKEN_ID   = 50256
phi_patches    = 49
vocab_size     = 51200
max_generate_length = 100
model_val_step      = 1000
model_log_step      = 100
model_save_step     = 100



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.60k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

In [7]:
torch.cuda.amp.autocast(enabled=True)

In [8]:
wandb.init(project  = "tsai_clip_phi2_project", name="finetune_QLoRA")

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


# 1 - DATALOADERS

In [9]:
# training data
csv_file = 'final_train_tokens.csv'
qa_dataset = pd.read_csv(csv_file)

In [10]:
train_dataloader = DataLoader(llavadataset(qa_dataset, phi_model_name,clip_model_name,tokenizer,processor),
                  collate_fn=collate_fn, batch_size=train_batch_size, num_workers = num_workers, shuffle=True, pin_memory=True)

In [11]:
file = open('sample_val_data.csv')
csvreader = csv.reader(file)
sample_val_data = []
for row in csvreader:
    sample_val_data.append(row)
print(sample_val_data[1])
file.close()

['http://images.cocodataset.org/train2017/000000027691.jpg', 'What are the colors of the dog in the image?', 'The dog in the image has black and brown fur.']


# 2 - MODELS

In [12]:
clip_model = CLIPVisionModel.from_pretrained(clip_model_name).to(device)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,)

phi_model = AutoModelForCausalLM.from_pretrained(
    phi_model_name,
    torch_dtype=torch.float32,
    quantization_config=bnb_config,
    trust_remote_code=True
)
phi_model.config.use_cache = False
projection = torch.nn.Linear(clip_embed, phi_embed).to(device)


pytorch_model.bin:   0%|          | 0.00/462M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


config.json:   0%|          | 0.00/863 [00:00<?, ?B/s]

configuration_phi.py:   0%|          | 0.00/9.26k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-2:
- configuration_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [16]:
print(phi_model)

PhiForCausalLM(
  (model): PhiModel(
    (embed_tokens): Embedding(51200, 2560)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x PhiDecoderLayer(
        (self_attn): PhiAttention(
          (q_proj): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (k_proj): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (v_proj): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (dense): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (rotary_emb): PhiRotaryEmbedding()
        )
        (mlp): PhiMLP(
          (activation_fn): NewGELUActivation()
          (fc1): Linear4bit(in_features=2560, out_features=10240, bias=True)
          (fc2): Linear4bit(in_features=10240, out_features=2560, bias=True)
        )
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (final_layern

In [13]:
lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "dense",
        "fc1",
        "fc2",
    ]
)
peft_model = peft.get_peft_model(phi_model, peft_config).to(device)
peft_model.print_trainable_parameters()

trainable params: 94,371,840 || all params: 2,874,055,680 || trainable%: 3.2835773035545364


In [14]:
# clip non trainable
for network in [clip_model]:
    for param in network.parameters():
        param.requires_grad_(False)

In [15]:
# check trainable paramaeters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"PEFT MODEL:{count_parameters(peft_model)}")
print(f"PROJECTION MODEL:{count_parameters(projection)}")
print(f"CLIP MODEL:{count_parameters(clip_model)}")
print(f"PHI MODEL:{count_parameters(phi_model)}")

PEFT MODEL:94371840
PROJECTION MODEL:1640960
CLIP MODEL:0
PHI MODEL:94371840


In [16]:
if os.path.isfile('model_chkpt/step2_projection.pth'):
    projection.load_state_dict(torch.load('./model_chkpt/ft_projection.pth'))
    peft_model.from_pretrained(phi_model,'./model_chkpt/qlora_adaptor')
    print("Loaded step2 checkpoint")

else:
    projection.load_state_dict(torch.load('./model_chkpt/clipphi_proj.pth'))
    print("Loaded step1 checkpoint")

Loaded step1 checkpoint


In [17]:
sample_val_data[100]

['http://images.cocodataset.org/train2017/000000548471.jpg',
 'What type of remote is the boy holding?',
 'The boy is holding a Nintendo Wii remote.']

# 3 - FUNCTIONS

In [18]:
# random validation prediction
def model_run_val(sample_val_data,max_generate_length=10):

    total_val_len = len(sample_val_data)
    random_val_datapoint = random.randrange(1,total_val_len) # 0 is header

    val_image_url = sample_val_data[random_val_datapoint][0]
    val_q = sample_val_data[random_val_datapoint][1]
    val_a = sample_val_data[random_val_datapoint][2]

    with torch.no_grad():
        image_load = Image.open(requests.get(val_image_url,stream=True).raw)
        image_processed = processor(images=image_load, return_tensors="pt").to(device)
        clip_val_outputs = clip_model(**image_processed).last_hidden_state[:,1:,:]
        val_image_embeds = projection(clip_val_outputs).to(torch.float16)



        img_token_tensor = torch.tensor(IMAGE_TOKEN_ID).to(device)
        img_token_embeds = peft_model.model.model.embed_tokens(img_token_tensor).unsqueeze(0).unsqueeze(0)

        val_q_tokenised = tokenizer(val_q, return_tensors="pt", return_attention_mask=False)['input_ids'].squeeze(0)
        val_q_embeds  = peft_model.model.model.embed_tokens(val_q_tokenised).unsqueeze(0)

        val_combined_embeds = torch.cat([val_image_embeds, img_token_embeds, val_q_embeds], dim=1) # 1, 69, 2560

        predicted_caption = torch.full((1,max_generate_length),50256)

        for g in range(max_generate_length):
            phi_output_logits = peft_model(inputs_embeds=val_combined_embeds)['logits'] # 4, 69, 51200
            predicted_word_token_logits = phi_output_logits[:, -1, :].unsqueeze(1) # 4,1,51200
            predicted_word_token   = torch.argmax(predicted_word_token_logits, dim = -1) # 4,1
            predicted_caption[:,g] = predicted_word_token.view(1,-1).to('cpu')
            next_token_embeds      = phi_model.model.embed_tokens(predicted_word_token) # 4,1,2560
            val_combined_embeds    = torch.cat([val_combined_embeds, next_token_embeds], dim=1)

        predicted_captions_decoded = tokenizer.batch_decode(predicted_caption,ignore_index = 50256)[0]

    print(f"Image: {val_image_url}")
    print(f"Question: {val_q}")
    print(f"Answer:   {val_a}")
    print(f"Model Predicted Ans: {predicted_captions_decoded}")

model_run_val(sample_val_data,max_generate_length=100)

Image: http://images.cocodataset.org/train2017/000000238154.jpg
Question: Is the bed made or unmade in the image?
Answer:   The bed in the image is unmade.
Model Predicted Ans: . a bed a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a


# 3 - TRAIN

In [19]:
import bitsandbytes as bnb

In [20]:
phi_optimizer        = bnb.optim.Adam8bit(peft_model.parameters(), lr=1e-6)
projection_optimizer = bnb.optim.Adam8bit(projection.parameters(), lr=1e-5)


In [21]:
step = 0
running_loss = 0.
projection.train()
peft_model.train()


for epoch in range(2):
    for batch_idx, (images,questions,answers) in enumerate(train_dataloader):

        # process input data
        batch_size = questions.size(0)
        questions  = questions.to(device)
        answers    = answers.to(device)

        # clip
        images = {'pixel_values': images.to(device)}
        clip_outputs  = clip_model(**images)
        images_embeds = clip_outputs.last_hidden_state[:,1:,:] # remove cls token

        # projection
        image_embeds  = projection(images_embeds)


        # embeds
        #print(questions.shape,answers.shape)
        img_token_tensor = torch.tensor(IMAGE_TOKEN_ID).repeat(batch_size, 1).to(device)
        img_token_embeds = peft_model.model.model.embed_tokens(img_token_tensor)
        questions_embed  = peft_model.model.model.embed_tokens(questions)

        # forward pass
        #print("***************")
        combined_embeds = torch.cat([image_embeds, img_token_embeds, questions_embed], dim=1) # 4, 69, 2560
        #print(f"combined_embeds shape{combined_embeds.shape}")
        phi_output_logits = peft_model(inputs_embeds=combined_embeds)['logits'] # 4, 69, 51200
        #print(f"phi_output_logits shape{phi_output_logits.shape}")
        #print(f"answers shape {answers.shape}")

        # take out the image embeddings
        phi_output_logits = phi_output_logits[:,images_embeds.shape[1] + 1 : ,:]
        #print(f"phi_output_logits after shape{phi_output_logits.shape}")
        phi_output_logits = phi_output_logits.reshape(-1,vocab_size)
        #print(f"phi_output_logits after shape{phi_output_logits.shape}")
        #print(f"answers after shape {answers.contiguous().view(-1).shape}")

        phi_optimizer.zero_grad()
        projection_optimizer.zero_grad()


        loss = F.cross_entropy(phi_output_logits, answers.contiguous().view(-1), ignore_index=EOS_TOKEN_ID,label_smoothing=0.1)

        # loss backprop
        loss.backward()
        phi_optimizer.step()
        projection_optimizer.step()



        if step % model_log_step == 0:
            print(f"Iteration {step}/{max_steps}, Loss: {loss.item()}")

        if step % model_val_step == 0:
            projection.eval()
            peft_model.eval()

            model_run_val(sample_val_data,max_generate_length)
            projection.train()
            peft_model.train()


        if step % model_save_step == 0:
            print("Saving Checkpoint")
            torch.save(projection.state_dict(),'./model_chkpt/ft_projection.pth')

            peft_model.save_pretrained('./model_chkpt/qlora_adaptor/', save_adapter=True, save_config=True)

        if step >= max_steps:
            print("Training finished.")
            break

        wandb.log({"step": step, "train_loss": loss.item()})
        step += 1

Iteration 0/10000, Loss: 5.142107009887695
Image: http://images.cocodataset.org/train2017/000000299923.jpg
Question: Is the door of the refrigerator open or closed?
Answer:   The door of the refrigerator is open, or ajar.
Model Predicted Ans: ....................

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.


Saving Checkpoint
Iteration 100/10000, Loss: 3.9319214820861816
Saving Checkpoint
Iteration 200/10000, Loss: 4.371220111846924
Saving Checkpoint
Iteration 300/10000, Loss: 4.394586563110352
Saving Checkpoint
Iteration 400/10000, Loss: 3.5000967979431152
Saving Checkpoint
Iteration 500/10000, Loss: 4.360044479370117
Saving Checkpoint
Iteration 600/10000, Loss: 3.4147374629974365
Saving Checkpoint
Iteration 700/10000, Loss: 3.0007400512695312
Saving Checkpoint
Iteration 800/10000, Loss: 3.8849291801452637
Saving Checkpoint
Iteration 900/10000, Loss: 3.958127737045288
Saving Checkpoint
Iteration 1000/10000, Loss: 3.6793971061706543
Image: http://image